In [71]:
#Importando bibliotecas
#Pandas é uma biblioteca para análise de dados em Python
#pyspark.sql.functions: É um módulo do PySpark que contém diversas funções para manipulação de dados em DataFrames.
#: Spark context  permite configurar e conectar-se a um cluster do Sparkrk, o SparkContext é necessário para criar uma sessão do Spark e trabalhar com RDDs
import pandas as pd
from pyspark.sql.functions import col, explode, sum
from pyspark import SparkContext

In [8]:
#Iniciando a sessão no spark
from pyspark.sql import SparkSession
sc = SparkContext
spark = SparkSession.builder.appName('Recomendacoes').getOrCreate()

In [10]:
#Carregando os conjuntos de dados pela função do spark para ler arquivos csv
movies = spark.read.csv("C:/Users/Dell/Desktop/filmes.csv",header=True)
ratings = spark.read.csv("C:/Users/Dell/Desktop/notas.csv",header=True)

<H2>Analise exploratoria</H2>

In [12]:
#Visualizando o conjunto de dados dos filmes
movies.show()

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
|     11|American Presiden...|Comedy|Drama|Romance|
|     12|Dracula: Dead and...|       Comedy|Horror|
|     13|        Balto (1995)|Adventure|Animati...|
|     14|        Nixon (1995)|               Drama|
|     15|Cutthroat Island ...|Action|Adventure|...|
|     16|       Casino (1995)|         Crime|Drama|
|     17|Sen

In [17]:
ratings.show()

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
|     1|     70|   3.0|964982400|
|     1|    101|   5.0|964980868|
|     1|    110|   4.0|964982176|
|     1|    151|   5.0|964984041|
|     1|    157|   5.0|964984100|
|     1|    163|   5.0|964983650|
|     1|    216|   5.0|964981208|
|     1|    223|   3.0|964980985|
|     1|    231|   5.0|964981179|
|     1|    235|   4.0|964980908|
|     1|    260|   5.0|964981680|
|     1|    296|   3.0|964982967|
|     1|    316|   3.0|964982310|
|     1|    333|   5.0|964981179|
|     1|    349|   4.0|964982563|
+------+-------+------+---------+
only showing top 20 rows



In [19]:
#Analise o tipo dos dados das variaveis
movies.printSchema()

root
 |-- movieId: string (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)



In [21]:
#Analise o tipo dos dados das variaveis
ratings.printSchema()

root
 |-- userId: string (nullable = true)
 |-- movieId: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- timestamp: string (nullable = true)



In [29]:
#Vendo a quantidade de linhas dos dois conjuntos de dados
print(f'a quantidade de linhas do dataset ratings é {ratings.count()} , e a quantidade de linhas do dataset de filmes é  {movies.count()}')


a quantidade de linhas do dataset ratings é 100836 , e a quantidade de linhas do dataset de filmes é  9742


In [33]:
#Vendo a quantidade de colunas dos dois conjuntos de dados
print(f'o dataset ratings possui {len(ratings.columns)} colunas, e o dataset filmes possui {len(movies.columns) }')

o dataset ratings possui 3 colunas, e o dataset filmes possui 3


In [23]:
#Vemos que algumas variaveis do dataset ratings não estão no formato correto, para realizar esse ajuste iremos realizar um casting nos dados e também
#Iremos deletar a coluna "timestamp" pois não iremos necessitar dela 

ratings = ratings.\
    withColumn('userId', col('userId').cast('integer')).\
    withColumn('movieId', col('movieId').cast('integer')).\
    withColumn('rating', col('rating').cast('float')).\
    drop('timestamp')
ratings.show()

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|      1|   4.0|
|     1|      3|   4.0|
|     1|      6|   4.0|
|     1|     47|   5.0|
|     1|     50|   5.0|
|     1|     70|   3.0|
|     1|    101|   5.0|
|     1|    110|   4.0|
|     1|    151|   5.0|
|     1|    157|   5.0|
|     1|    163|   5.0|
|     1|    216|   5.0|
|     1|    223|   3.0|
|     1|    231|   5.0|
|     1|    235|   4.0|
|     1|    260|   5.0|
|     1|    296|   3.0|
|     1|    316|   3.0|
|     1|    333|   5.0|
|     1|    349|   4.0|
+------+-------+------+
only showing top 20 rows



In [63]:
# Contando o número de userIds distintos e movieIds distintos
num_users = ratings.select("userId").distinct().count()
num_movies = ratings.select("movieId").distinct().count()

print(f'o número de usuários distintos é {num_users} , o número de filmes distintos é {num_movies}')

o número de usuários distintos é 610 , o número de filmes distintos é 9724


In [61]:
# Definindo o denominador igual ao número de usuários multiplicado pelo número de filmes
denominador = num_users * num_movies
denominador

5931640

In [65]:
#Calculando a esparsidade dos dados do dataset ratings, é nesse dataset que vamos aplicar o algoritmo de machine learning
#Esparsidade em análise de dados refere-se à proporção de valores nulos ou zero em um conjunto de dados
#Se a esparsidade for próxima de 1, significa que a matriz tem muitos valores zero (ou nulos) e é altamente esparsa.

# Contando o número de userIds distintos e movieIds distintos
esparsidade = (1.0 - (numerator *1.0)/denominator)*100
print("O dataframe Ratings é ", "%.2f" % esparsidade + "% vazio.")

O dataframe Ratings é  98.30% vazio.


In [81]:
#Calculando a quantidade de valores null nos dataset ratings
null_ratings = ratings.select([sum(col(c).isNull().cast("int")).alias(c) for c in ratings.columns])
null_ratings.show()

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     0|      0|     0|
+------+-------+------+



In [85]:
#Calculando a quantidade de valores null nos dataset ratings
null_movies = movies.select([sum(col(c).isNull().cast("int")).alias(c) for c in movies.columns])
null_movies.show()

+-------+-----+------+
|movieId|title|genres|
+-------+-----+------+
|      0|    0|     0|
+-------+-----+------+



In [87]:
#Agrupando dados por userId, contando por ratings
userId_ratings = ratings.groupBy("userId").count().orderBy('count', ascending=False)
userId_ratings.show()

+------+-----+
|userId|count|
+------+-----+
|   414| 2698|
|   599| 2478|
|   474| 2108|
|   448| 1864|
|   274| 1346|
|   610| 1302|
|    68| 1260|
|   380| 1218|
|   606| 1115|
|   288| 1055|
|   249| 1046|
|   387| 1027|
|   182|  977|
|   307|  975|
|   603|  943|
|   298|  939|
|   177|  904|
|   318|  879|
|   232|  862|
|   480|  836|
+------+-----+
only showing top 20 rows



In [89]:
#Agrupando dados por userId, contando por ratings
movieId_ratings = ratings.groupBy("movieId").count().orderBy('count', ascending=False)
movieId_ratings.show()

+-------+-----+
|movieId|count|
+-------+-----+
|    356|  329|
|    318|  317|
|    296|  307|
|    593|  279|
|   2571|  278|
|    260|  251|
|    480|  238|
|    110|  237|
|    589|  224|
|    527|  220|
|   2959|  218|
|      1|  215|
|   1196|  211|
|     50|  204|
|   2858|  204|
|     47|  203|
|    780|  202|
|    150|  201|
|   1198|  200|
|   4993|  198|
+-------+-----+
only showing top 20 rows



<H2>Construindo o modelo de Recomendação</H2>

In [92]:
# Importação de bibliotecas para criar os algoritmos
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

In [96]:
# Criando conjuntos de treino e teste
(train, test) = ratings.randomSplit([0.8, 0.2], seed = 1234)

# Criando o modelo de Recomendação
als = ALS(userCol="userId", itemCol="movieId", ratingCol="rating", nonnegative = True, implicitPrefs = False, coldStartStrategy="drop")

# Confirmando se o modelo foi criado
type(als)

pyspark.ml.recommendation.ALS

<H2>Ajustando o modelo</h2>

In [101]:
# Importando novas bibliotecas
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# Adicionando hiperparâmetros e seus respectivos valores em param_grid,para ajustes de hiperparâmetros do modelo ALS
param_grid = ParamGridBuilder() \
            .addGrid(als.rank, [10, 50, 100, 150]) \
            .addGrid(als.regParam, [.01, .05, .1, .15]) \
            .build()
            #             .addGrid(als.maxIter, [5, 50, 100, 200]) \

           
# Defina o avaliador como RMSE e imprime o comprimento do avaliador
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction") 
print ("Número de modelos a serem testados: ", len(param_grid))

Número de modelos a serem testados:  16


In [103]:
#Criando o pipeline de validação cruzada
cv = CrossValidator(estimator=als, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=5)

# Confirm cv was built
print(cv)

CrossValidator_5e18caa8f5a4


In [105]:
#Treinando o modelo de validação cruzada
model = cv.fit(train)

#retornando o melhor modelo que foi ajustado durante o processo de validação cruzada
best_model = model.bestModel

In [106]:
# Print melhor modelo
print(type(best_model))


print("**Best Model**")

# # Print "Rank" número de fatores latentes no modelo ALS.
print("  Rank:", best_model._java_obj.parent().getRank())

# Print "MaxIter" número máximo de iterações que o modelo ALS faz durante o treinamento
print("  MaxIter:", best_model._java_obj.parent().getMaxIter())

# Print "RegParam" controla a penalização para evitar overfitting
print("  RegParam:", best_model._java_obj.parent().getRegParam())

<class 'pyspark.ml.recommendation.ALSModel'>
**Best Model**
  Rank: 150
  MaxIter: 10
  RegParam: 0.15


In [110]:
# Vendo as predições
test_predictions = best_model.transform(test)
RMSE = evaluator.evaluate(test_predictions)
print(RMSE)

0.8689288255111681


In [111]:
test_predictions.show()

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|   580|   1580|   4.0| 3.4556541|
|   580|  44022|   3.5| 3.2440484|
|   597|    471|   2.0|  4.199578|
|   108|   1959|   5.0| 3.8372588|
|   368|   2122|   2.0| 1.8555024|
|   436|    471|   3.0| 3.6066322|
|   587|   1580|   4.0| 3.8765278|
|    27|   1580|   3.0|  3.344146|
|   606|   1580|   2.5| 3.1791172|
|   606|  44022|   4.0| 2.8043003|
|    91|   2122|   4.0| 2.3345869|
|   157|   3175|   2.0| 3.4112093|
|   232|   1580|   3.5|  3.370875|
|   232|  44022|   3.0| 3.1538184|
|   246|   1645|   4.0| 3.7365985|
|   599|   2366|   3.0| 2.9035661|
|   111|   1088|   3.0| 3.2689178|
|   111|   3175|   3.5| 3.0905557|
|    47|   1580|   1.5| 2.6785266|
|   140|   1580|   3.0|  3.371964|
+------+-------+------+----------+
only showing top 20 rows



<h2>Fazendo recomendações</h2>

In [114]:
# Gerando n recomendações para todos os usuários
nrecommendations = best_model.recommendForAllUsers(10)
nrecommendations.limit(10).show()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|     1|[{3379, 5.697951}...|
|     2|[{131724, 4.79383...|
|     3|[{6835, 4.8520045...|
|     4|[{3851, 4.848169}...|
|     5|[{5490, 4.545368}...|
|     6|[{42730, 4.717121...|
|     7|[{33649, 4.406992...|
|     8|[{3379, 4.6708913...|
|     9|[{5490, 4.837154}...|
|    10|[{71579, 4.533971...|
+------+--------------------+



In [115]:
nrecommendations = nrecommendations\
    .withColumn("rec_exp", explode("recommendations"))\
    .select('userId', col("rec_exp.movieId"), col("rec_exp.rating"))

nrecommendations.limit(10).show()

+------+-------+---------+
|userId|movieId|   rating|
+------+-------+---------+
|     1|   3379| 5.697951|
|     1|  33649|5.5743775|
|     1|   5490|  5.53618|
|     1| 171495| 5.405559|
|     1|   5915| 5.395145|
|     1|   5416|5.3816915|
|     1|   5328|5.3816915|
|     1|   3951|5.3816915|
|     1|  78836| 5.357355|
|     1| 184245| 5.302923|
+------+-------+---------+



In [116]:
#Mesclando o nome e os gêneros do filme à matriz de recomendação ver se está fazendo sentido as recomendações:
nrecommendations.join(movies, on='movieId').filter('userId = 100').show()

+-------+------+---------+--------------------+--------------------+
|movieId|userId|   rating|               title|              genres|
+-------+------+---------+--------------------+--------------------+
|  67618|   100|5.1082697|Strictly Sexual (...|Comedy|Drama|Romance|
|  33649|   100| 4.990537|  Saving Face (2004)|Comedy|Drama|Romance|
|   3379|   100| 4.986049| On the Beach (1959)|               Drama|
|  74282|   100|4.8992786|Anne of Green Gab...|Children|Drama|Ro...|
| 184245|   100|4.8964562|De platte jungle ...|         Documentary|
| 179135|   100|4.8964562|Blue Planet II (2...|         Documentary|
| 138966|   100|4.8964562|Nasu: Summer in A...|           Animation|
| 117531|   100|4.8964562|    Watermark (2014)|         Documentary|
|  86237|   100|4.8964562|  Connections (1978)|         Documentary|
|  84273|   100|4.8964562|Zeitgeist: Moving...|         Documentary|
+-------+------+---------+--------------------+--------------------+



In [118]:
ratings.join(movies, on='movieId').filter('userId = 100').sort('rating', ascending=False).limit(10).show()

+-------+------+------+--------------------+--------------------+
|movieId|userId|rating|               title|              genres|
+-------+------+------+--------------------+--------------------+
|   1101|   100|   5.0|      Top Gun (1986)|      Action|Romance|
|   1958|   100|   5.0|Terms of Endearme...|        Comedy|Drama|
|   2423|   100|   5.0|Christmas Vacatio...|              Comedy|
|   4041|   100|   5.0|Officer and a Gen...|       Drama|Romance|
|   5620|   100|   5.0|Sweet Home Alabam...|      Comedy|Romance|
|    368|   100|   4.5|     Maverick (1994)|Adventure|Comedy|...|
|    934|   100|   4.5|Father of the Bri...|              Comedy|
|    539|   100|   4.5|Sleepless in Seat...|Comedy|Drama|Romance|
|     16|   100|   4.5|       Casino (1995)|         Crime|Drama|
|    553|   100|   4.5|    Tombstone (1993)|Action|Drama|Western|
+-------+------+------+--------------------+--------------------+

